In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import doctest

In [3]:
import abc


class AutoStorage:
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = f'_{prefix}#{index}'
        cls.__counter += 1

    def __get__(self, obj, objtype):
        if obj is None:
            return self
        else:
            return getattr(obj, self.storage_name)

    def __set__(self, obj, value):
        setattr(obj, self.storage_name, value)


class Validated(AutoStorage):
    def __set__(self, obj, value):
        value = self.validate(obj, value)
        super().__set__(obj, value)

    @abc.abstractmethod
    def validate(self, obj, value):
        pass


class Quantity(Validated):
    def validate(self, obj, value):
        if value > 0:
            return value

        raise ValueError('value must be > 0')


class NonBlank(Validated):
    def validate(self, obj, value):
        value = value.strip()
        if len(value) > 0:
            return value

        raise ValueError('value cannot be empty or blank')


class LineItem:
    description = NonBlank()
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price


"""

>>> br_nuts = LineItem('Brazil nuts', 10, 34.95)
>>> br_nuts.subtotal()
349.5
>>> vars(br_nuts)
{'_NonBlank#0': 'Brazil nuts', '_Quantity#0': 10, '_Quantity#1': 34.95}
>>> LineItem.price  # doctest: +ELLIPSIS
<__main__.Quantity object at ...>
>>> br_nuts.price = -3
Traceback (most recent call last):
    ...
ValueError: value must be > 0
>>> br_nuts.description = '  '
Traceback (most recent call last):
    ...
ValueError: value cannot be empty or blank
"""

doctest.testmod()

TestResults(failed=0, attempted=6)